In [1]:
# use seastar Kernel on adrien local laptop

# import os
import xarray as xr
import numpy as np
# import numpy.ma as ma
import matplotlib.pyplot as plt
import cartopy.crs as ccrs # => to add in Seastar env
import seastar
from seastar.utils.tools import dotdict

from scipy import interpolate, stats # => to add in Seastar env ?
from scipy.optimize import least_squares

# from seastar.gmfs.doppler import *

In [2]:
# % Antennae
# % Main parameters from the EE11 proposal                                        
                                                                                
# % geometry                                                                      
inc_angle_mid_propa  = [20.0; 28.4; 33.4];                                      
inc_angle_sqt_propa  = [31.5; 36.5; 40.0];                                      
squint_ground_propa  = [37.8; 52.2]; % from the North/sat direction; 90-x for sqt from mid
                                                                                
# % Resolution                                                                    
mid_range_resolution = 28; % in meter                                           
mid_azimuth_resolution = 20; % in meter                                         
sqt_range_resolution = 28; % in meter                                           
sqt_azimuth_resolution = 20; % in meter                                         
                                                                                
# % Noise                                                                         
NESZ_sqt_NearSwath_propa   = [-26; -28; -26]; % @inci [20; 24.2; 28.4-]         
NESZ_sqt_FarSwath_propa    = [-27; -28; -27]; % @inci [28.4+; 30.9; 33.4]       
NESZ_mid_NearSwath_propa   = [-20; -26; -20]; % @inci [20; 24.2; 28.4-]         
NESZ_mid_FarSwath_propa   = [-26; -28; -26]; % @inci [28.4+; 30.9; 33.4]        
                                                                                
# % Interpolation over squint inci angle                                          
inc_angle_sqt  = [31.5; 34.0; 36.0; 37.0; 38.5; 40.0];                          
inc_angle_mid  = interp1(inc_angle_sqt_propa, inc_angle_mid_propa, inc_angle_sqt, 'linear');
squint_ground  = interp1(inc_angle_sqt_propa([1, end]), squint_ground_propa, inc_angle_sqt, 'linear');
NESZ_sqt       = interp1([linspace(inc_angle_sqt_propa(1), inc_angle_sqt_propa(2)-0.001, 3),...
                           linspace(inc_angle_sqt_propa(2), inc_angle_sqt_propa(3), 3)],...
                           [NESZ_sqt_NearSwath_propa', NESZ_sqt_FarSwath_propa'],...
                           inc_angle_sqt, 'linear');                            
NESZ_mid       = interp1([linspace(inc_angle_sqt_propa(1), inc_angle_sqt_propa(2)-0.001, 3),...
                           linspace(inc_angle_sqt_propa(2), inc_angle_sqt_propa(3), 3)],...
                           [NESZ_mid_NearSwath_propa', NESZ_mid_FarSwath_propa'],...
                           inc_angle_sqt, 'linear');                            
                                                                                
# % Along track Interferometry Baseline                                           
Vp = 7.4 * 1000; % in m/s                                                       
Baseline_phy = 15; % in meter                                                   
                                                                                
# % NRCS GMF name                                                                 
NRCS_GMF.name = 'NSCAT1';     

SyntaxError: invalid syntax (3239496455.py, line 5)

In [2]:
inst = xr.Dataset(
    data_vars=dict(
        CentralWavenumber=( [],270 ),# TODO this should be calculated if needed from CentralFreq, not put as an input
        CentralFreq=( [], 13.5 * 10**9 ),
        IncidenceAngleImage=( ['across','Antenna'], np.full([5,4], 30) ),
        AntennaAzimuthImage=(['across', 'Antenna'],
                           np.stack((np.full([5], 45),
                                     np.full([5], 90),
                                     np.full([5], 90),
                                     np.full([5], 135)
                                     ), axis=-1
                                    )
                           ),
            Polarization=(['across','Antenna'],
                          np.stack((np.full([5,], 'VV'),
                                    np.full([5,], 'VV'),
                                    np.full([5,], 'HH'),
                                    np.full([5,], 'VV')
                                    ), axis=-1
                                   )
                          ),
    ),
    coords=dict(
            across=np.arange(0,5),
            Antenna=['Fore','MidV','MidH','Aft'],
        ),
)
inst = inst.set_coords([
    'CentralWavenumber',
    'CentralFreq',
#     'IncidenceAngleImage',
#     'AntennaAzimuthImage',
#     'Polarization', 
])
inst

<xarray.Dataset>
Dimensions:              (across: 5, Antenna: 4)
Coordinates:
    CentralWavenumber    int64 270
    CentralFreq          float64 1.35e+10
  * across               (across) int64 0 1 2 3 4
  * Antenna              (Antenna) <U4 'Fore' 'MidV' 'MidH' 'Aft'
Data variables:
    IncidenceAngleImage  (across, Antenna) int64 30 30 30 30 30 ... 30 30 30 30
    AntennaAzimuthImage  (across, Antenna) int64 45 90 90 135 ... 45 90 90 135
    Polarization         (across, Antenna) <U2 'VV' 'VV' 'HH' ... 'VV' 'HH' 'VV'

In [3]:
# along = xr.Dataset(coords={'along': np.arange(0,6)})

In [4]:
# level1 = inst * along # cf https://github.com/pydata/xarray/discussions/6163

In [5]:
geo = xr.Dataset(
        data_vars=dict(
            WindSpeed=(['across', 'along'], np.full([5, 6], 10)),
            WindDirection=(['across', 'along'], np.full([5, 6], 150)),
            CurrentVelocity=(['across', 'along'], np.full([5, 6], 1)),
            CurrentDirection=(['across', 'along'], np.full([5, 6], 150)),
        ),
        coords=dict(
            across=np.arange(0, 5),
            along=np.arange(0, 6),
        ),
    )
geo

<xarray.Dataset>
Dimensions:           (across: 5, along: 6)
Coordinates:
  * across            (across) int64 0 1 2 3 4
  * along             (along) int64 0 1 2 3 4 5
Data variables:
    WindSpeed         (across, along) int64 10 10 10 10 10 10 ... 10 10 10 10 10
    WindDirection     (across, along) int64 150 150 150 150 ... 150 150 150 150
    CurrentVelocity   (across, along) int64 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
    CurrentDirection  (across, along) int64 150 150 150 150 ... 150 150 150 150

In [6]:
truth = xr.broadcast(inst, geo)[0]
truth

<xarray.Dataset>
Dimensions:              (across: 5, Antenna: 4, along: 6)
Coordinates:
    CentralFreq          float64 1.35e+10
    CentralWavenumber    int64 270
  * across               (across) int64 0 1 2 3 4
  * Antenna              (Antenna) <U4 'Fore' 'MidV' 'MidH' 'Aft'
  * along                (along) int64 0 1 2 3 4 5
Data variables:
    IncidenceAngleImage  (across, Antenna, along) int64 30 30 30 30 ... 30 30 30
    AntennaAzimuthImage  (across, Antenna, along) int64 45 45 45 ... 135 135 135
    Polarization         (across, Antenna, along) <U2 'VV' 'VV' ... 'VV' 'VV'

In [7]:
gmf=dotdict({
    'nrcs': dotdict({'name': 'nscat4ds'}),
    'doppler': dotdict({'name': 'mouche12'}),
})

In [8]:
# TODO add the broadcast to create truth in the truth_fct
truth = seastar.performance.scene_generation.truth_fct(geo, truth, gmf)
truth

<xarray.Dataset>
Dimensions:              (across: 5, Antenna: 4, along: 6)
Coordinates:
    IncidenceAngleImage  (across, Antenna, along) int64 30 30 30 30 ... 30 30 30
    AntennaAzimuthImage  (across, Antenna, along) int64 45 45 45 ... 135 135 135
    Polarization         (across, Antenna, along) <U2 'VV' 'VV' ... 'VV' 'VV'
    CentralFreq          float64 1.35e+10
    CentralWavenumber    int64 270
  * across               (across) int64 0 1 2 3 4
  * Antenna              (Antenna) <U4 'Fore' 'MidV' 'MidH' 'Aft'
  * along                (along) int64 0 1 2 3 4 5
Data variables:
    WindSpeed            (across, along) int64 10 10 10 10 10 ... 10 10 10 10 10
    WindDirection        (across, along) int64 150 150 150 150 ... 150 150 150
    CurrentVelocity      (across, along) int64 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1
    CurrentDirection     (across, along) int64 150 150 150 150 ... 150 150 150
    Sigma0               (Antenna, across, along) float64 0.07121 ... 0.1666
    RVL                  (Antenna, across, along) float64 0.06358 ... -0.5307

In [9]:
# should build a class uncertainty, to check the inputs and calculate the other ones

# uncertainty=dotdict({ # should build a class uncertainty, to check the inputs and calculate the other ones
# #     'N': float, ndarray,
# #     'NESZ': float, ndarray,
#     'Kp': [0.03, 0.03, 0.04, 0.04],  # float, ndarray,
# #     'Kp_gen': float, ndarray,
# #     'Sigma0': # float, ndarray, = truth * Kp
#     'RVL': 0.05, # projected on the surface m/s
# })
# # for SEASTAR EE1 proposal it was used NESZ + N

uncertainty_in = xr.Dataset(
    data_vars=dict(
        Kp=( ['Antenna'], np.array([0.03, 0.05, 0.05, 0.03])),
        RVL=( ['Antenna'], np.array([0.05, 0.20, 0.20, 0.05])),
    ),
    coords=dict(
            Antenna=['Fore','MidV','MidH','Aft'],
        ),
)
uncertainty_in

<xarray.Dataset>
Dimensions:  (Antenna: 4)
Coordinates:
  * Antenna  (Antenna) <U4 'Fore' 'MidV' 'MidH' 'Aft'
Data variables:
    Kp       (Antenna) float64 0.03 0.05 0.05 0.03
    RVL      (Antenna) float64 0.05 0.2 0.2 0.05

In [10]:
[uncertainty, noise] = seastar.performance.scene_generation.uncertainty_fct(truth, uncertainty_in)
uncertainty, noise

To Be Done - uncertainty function


(<xarray.Dataset>
 Dimensions:              (Antenna: 4, across: 5, along: 6)
 Coordinates:
   * Antenna              (Antenna) <U4 'Fore' 'MidV' 'MidH' 'Aft'
     IncidenceAngleImage  (across, Antenna, along) int64 30 30 30 30 ... 30 30 30
     AntennaAzimuthImage  (across, Antenna, along) int64 45 45 45 ... 135 135 135
     Polarization         (across, Antenna, along) <U2 'VV' 'VV' ... 'VV' 'VV'
     CentralFreq          float64 1.35e+10
     CentralWavenumber    int64 270
   * across               (across) int64 0 1 2 3 4
   * along                (along) int64 0 1 2 3 4 5
 Data variables:
     Kp                   (Antenna) float64 0.03 0.05 0.05 0.03
     RVL                  (Antenna) float64 0.05 0.2 0.2 0.05
     Sigma0               (Antenna, across, along) float64 0.002136 ... 0.004999,
 <xarray.Dataset>
 Dimensions:              (Antenna: 4, across: 5, along: 6)
 Coordinates:
   * Antenna              (Antenna) <U4 'Fore' 'MidV' 'MidH' 'Aft'
     IncidenceAngleImage  (acros

In [12]:
level1 = seastar.performance.scene_generation.noise_generation(truth, noise)

In [13]:
level1

<xarray.Dataset>
Dimensions:              (Antenna: 4, across: 5, along: 6)
Coordinates:
  * Antenna              (Antenna) <U4 'Fore' 'MidV' 'MidH' 'Aft'
    IncidenceAngleImage  (across, Antenna, along) int64 30 30 30 30 ... 30 30 30
    AntennaAzimuthImage  (across, Antenna, along) int64 45 45 45 ... 135 135 135
    Polarization         (across, Antenna, along) <U2 'VV' 'VV' ... 'VV' 'VV'
    CentralFreq          float64 1.35e+10
    CentralWavenumber    int64 270
  * across               (across) int64 0 1 2 3 4
  * along                (along) int64 0 1 2 3 4 5
Data variables:
    Sigma0               (Antenna, across, along) float64 0.07471 ... 0.1728
    RVL                  (Antenna, across, along) float64 -0.02231 ... -0.5225

In [ ]:
%%time
##### WARNING ####### Take Time to compute #######
ambiguity = 'sort_by_cost'
level2, lmout = seastar.retrieval.level2.wind_current_retrieval(level1, noise, gmf, ambiguity)

To Be Done find_initial_value
To Be Done - ambiguity_sort_by_cost
To Be Done - solve_ambiguity
To Be Done find_initial_value
To Be Done - ambiguity_sort_by_cost
To Be Done - solve_ambiguity
To Be Done find_initial_value
To Be Done - ambiguity_sort_by_cost
To Be Done - solve_ambiguity
To Be Done find_initial_value
To Be Done - ambiguity_sort_by_cost
To Be Done - solve_ambiguity
To Be Done find_initial_value
To Be Done - ambiguity_sort_by_cost
To Be Done - solve_ambiguity
To Be Done find_initial_value
To Be Done - ambiguity_sort_by_cost
To Be Done - solve_ambiguity
To Be Done find_initial_value
To Be Done - ambiguity_sort_by_cost
To Be Done - solve_ambiguity
To Be Done find_initial_value
To Be Done - ambiguity_sort_by_cost
To Be Done - solve_ambiguity
To Be Done find_initial_value
To Be Done - ambiguity_sort_by_cost
To Be Done - solve_ambiguity
To Be Done find_initial_value
To Be Done - ambiguity_sort_by_cost
To Be Done - solve_ambiguity
To Be Done find_initial_value
To Be Done - ambigui

In [ ]:
level2

In [40]:
# Add NaN for RVL for the mid antennas
level1.RVL[1,:,:] = np.full([5,6], np.nan)
level1.RVL[2,:,:] = np.full([5,6], np.nan)


In [41]:
model_rvl_list = [None] * level1.Antenna.size
for aa, ant in enumerate(level1.Antenna.data):
    model_rvl_list[aa] = ss.gmfs.doppler.compute_total_surface_motion(level1.sel(Antenna=ant), geo, gmf='mouche12')

model = level1.drop_vars([var for var in level1.data_vars])
model['RVL'] = xr.concat(model_rvl_list, dim='Antenna')
model['Sigma0'] = ss.gmfs.nrcs.compute_nrcs(level1, geo, gmf=gmf.nrcs)

In [42]:
noise = level1.drop_vars([var for var in level1.data_vars])
# noise = level1.drop_vars(['Sigma0','dsig0','RVL','drvl'])
noise['Sigma0'] = level1.Sigma0*0.05
noise['RVL'] = level1.RVL*0.05

In [43]:
res = (model-level1)/noise

In [44]:
np.concatenate(
    (res.Sigma0.sel(across=1, along=1).data, res.RVL.sel(across=1, along=1).data)
)
    

array([-0.01998002, -0.01998002, -0.01998002, -0.01998002, -0.01998002,
               nan,         nan, -0.01998002])

# Test Fun_residual

In [45]:
sl1 = level1.sel(across=1, along=1)
sn = noise.sel(across=1, along=1)

In [46]:
ss.retrieval.cost_function.fun_residual([5,5,1,1], sl1, sn, gmf)

array([   9.01895349,   12.1681688 ,   12.60884007,   16.59575996,
       -691.66077601,    0.        ,    0.        ,   17.09819907])

### test fun_residual with 2D fields

In [47]:
[geo['U'], geo['V']] = ss.utils.tools.windSpeedDir2UV(geo.WindSpeed, geo.WindDirection)
[geo['C_U'], geo['C_V']] = ss.utils.tools.currentVelDir2UV(geo.CurrentVelocity, geo.CurrentDirection)

In [48]:
geo

<xarray.Dataset>
Dimensions:           (across: 5, along: 6)
Coordinates:
  * across            (across) int64 0 1 2 3 4
  * along             (along) int64 0 1 2 3 4 5
Data variables:
    WindSpeed         (across, along) int64 10 10 10 10 10 10 ... 10 10 10 10 10
    WindDirection     (across, along) int64 150 150 150 150 ... 150 150 150 150
    CurrentVelocity   (across, along) int64 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
    CurrentDirection  (across, along) int64 150 150 150 150 ... 150 150 150 150
    U                 (across, along) float64 -5.0 -5.0 -5.0 ... -5.0 -5.0 -5.0
    V                 (across, along) float64 8.66 8.66 8.66 ... 8.66 8.66 8.66
    C_U               (across, along) float64 0.5 0.5 0.5 0.5 ... 0.5 0.5 0.5
    C_V               (across, along) float64 -0.866 -0.866 ... -0.866 -0.866

In [49]:
level1

<xarray.Dataset>
Dimensions:              (across: 5, along: 6, Antenna: 4)
Coordinates:
    CentralWavenumber    int64 270
    CentralFreq          float64 1.35e+10
    IncidenceAngleImage  (across, along, Antenna) int64 30 30 30 30 ... 30 30 30
    AntennaAzimuthImage  (across, along, Antenna) int64 45 90 90 ... 90 90 135
    Polarization         (across, along, Antenna) <U2 'VV' 'VV' ... 'HH' 'VV'
  * across               (across) int64 0 1 2 3 4
  * along                (along) int64 0 1 2 3 4 5
  * Antenna              (Antenna) <U4 'Fore' 'MidV' 'MidH' 'Aft'
Data variables:
    Sigma0               (Antenna, across, along) float64 0.07128 ... 0.1668
    WASV                 (Antenna, across, along) float64 0.3227 ... -1.498
    RVL                  (Antenna, across, along) float64 0.06364 ... -0.5313

In [50]:
results = ss.retrieval.cost_function.fun_residual([geo.U.data, geo.V.data, geo.C_U.data, geo.C_V.data], level1, noise, gmf)

In [51]:
results = ss.retrieval.cost_function.fun_residual([geo.U, geo.V, geo.C_U, geo.C_V], level1, noise, gmf)

In [52]:
results

array([[[-3.04975857, -3.04975857, -3.04975857, -3.04975857,
         -3.04975857, -3.04975857],
        [-3.04975857, -3.04975857, -3.04975857, -3.04975857,
         -3.04975857, -3.04975857],
        [-3.04975857, -3.04975857, -3.04975857, -3.04975857,
         -3.04975857, -3.04975857],
        [-3.04975857, -3.04975857, -3.04975857, -3.04975857,
         -3.04975857, -3.04975857],
        [-3.04975857, -3.04975857, -3.04975857, -3.04975857,
         -3.04975857, -3.04975857]],

       [[-3.25064737, -3.25064737, -3.25064737, -3.25064737,
         -3.25064737, -3.25064737],
        [-3.25064737, -3.25064737, -3.25064737, -3.25064737,
         -3.25064737, -3.25064737],
        [-3.25064737, -3.25064737, -3.25064737, -3.25064737,
         -3.25064737, -3.25064737],
        [-3.25064737, -3.25064737, -3.25064737, -3.25064737,
         -3.25064737, -3.25064737],
        [-3.25064737, -3.25064737, -3.25064737, -3.25064737,
         -3.25064737, -3.25064737]],

       [[-3.27504276, -3.2

In [53]:
ss.retrieval.cost_function.fun_residual([geo.U, geo.V, geo.C_U, geo.C_V], level1, noise, gmf).shape

(8, 5, 6)

# Test least_squares with fun_residual

In [54]:
opt = {
        'method': 'trf', # Trust Region Reflective algorithm, particularly suitable for large sparse problems with bounds. Generally robust method.
        'xtol':  1e-1, # Tolerance for termination by the change of the independent variables
        'x_scale': [7,7,.5,.5], # Characteristic scale of each variable.
        'bounds': ([-30,-30,-5,-5], [30,30,5,5]),
    }
x0_variables = [4,4,0,0]

In [55]:
lmout = least_squares(
    ss.retrieval.cost_function.fun_residual,
    x0_variables,
    args=(sl1, sn, gmf),
    **opt
)

In [56]:
lmout

 active_mask: array([0, 0, 0, 0])
        cost: 359.2870934440441
         fun: array([-5.84578029,  0.84953228,  3.50291906, 13.55825463,  7.28518626,
        0.        ,  0.        , 20.84487192])
        grad: array([-1.11574674e-01,  1.29430683e+01, -1.06386311e+03, -2.18671347e+03])
         jac: array([[-5.87770054e+00, -3.71198733e+00,  5.87770021e+00,
         3.71198744e+00],
       [-4.73286728e+00, -1.55214232e+00,  4.73286674e+00,
         1.55214188e+00],
       [-3.78176642e+00, -1.51455966e+00,  3.78176609e+00,
         1.51455951e+00],
       [-1.26884883e+00, -1.57278294e-01,  1.26884854e+00,
         1.57278061e-01],
       [ 0.00000000e+00,  0.00000000e+00, -2.22214298e+02,
        -2.22214293e+02],
       [ 0.00000000e+00,  0.00000000e+00, -0.00000000e+00,
        -0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00, -0.00000000e+00,
        -0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  2.66203537e+01,
        -2.66203547e+01]])
     message: '`

# Test find_minima

In [57]:
level1

<xarray.Dataset>
Dimensions:              (across: 5, along: 6, Antenna: 4)
Coordinates:
    CentralWavenumber    int64 270
    CentralFreq          float64 1.35e+10
    IncidenceAngleImage  (across, along, Antenna) int64 30 30 30 30 ... 30 30 30
    AntennaAzimuthImage  (across, along, Antenna) int64 45 90 90 ... 90 90 135
    Polarization         (across, along, Antenna) <U2 'VV' 'VV' ... 'HH' 'VV'
  * across               (across) int64 0 1 2 3 4
  * along                (along) int64 0 1 2 3 4 5
  * Antenna              (Antenna) <U4 'Fore' 'MidV' 'MidH' 'Aft'
Data variables:
    Sigma0               (Antenna, across, along) float64 0.07128 ... 0.1668
    WASV                 (Antenna, across, along) float64 0.3227 ... -1.498
    RVL                  (Antenna, across, along) float64 0.06364 ... -0.5313

In [58]:
L1stack = level1.stack(z=("across", "along"))

In [59]:
noise

<xarray.Dataset>
Dimensions:              (across: 5, along: 6, Antenna: 4)
Coordinates:
    CentralWavenumber    int64 270
    CentralFreq          float64 1.35e+10
    IncidenceAngleImage  (across, along, Antenna) int64 30 30 30 30 ... 30 30 30
    AntennaAzimuthImage  (across, along, Antenna) int64 45 90 90 ... 90 90 135
    Polarization         (across, along, Antenna) <U2 'VV' 'VV' ... 'HH' 'VV'
  * across               (across) int64 0 1 2 3 4
  * along                (along) int64 0 1 2 3 4 5
  * Antenna              (Antenna) <U4 'Fore' 'MidV' 'MidH' 'Aft'
Data variables:
    Sigma0               (Antenna, across, along) float64 0.003564 ... 0.008339
    RVL                  (Antenna, across, along) float64 0.003182 ... -0.02656

In [60]:
Nstack = noise.stack(z=("across", "along"))

In [61]:
gmf

{'nrcs': {'name': 'nscat4ds'}, 'doppler': {'name': 'mouche12'}}

In [62]:
lmoutmap = [None] * L1stack.z.size

In [30]:
for ii, zindex in enumerate(L1stack.z.data):
    print(ii)
    sL1 = L1stack.sel(z=zindex)
    sN = Nstack.sel(z=zindex)
    lmoutmap[ii] = ss.retrieval.cost_function.find_minima(sL1, sN, gmf)

0


KeyboardInterrupt: 

In [31]:
lmmap = xr.concat(lmoutmap, dim='z')

TypeError: can only concatenate xarray Dataset and DataArray objects, got <class 'NoneType'>

In [32]:
lmmap['z'] = L1stack.z

NameError: name 'lmmap' is not defined

In [33]:
sol = lmmap.unstack(dim='z')

NameError: name 'lmmap' is not defined

In [34]:
sol

NameError: name 'sol' is not defined

In [35]:
s1 = sol.isel(along=0, across=0).sortby('cost')

NameError: name 'sol' is not defined

In [36]:
s1.x.isel(Ambiguities=0).sel(x_variables='c_v')

NameError: name 's1' is not defined

# Test wind_current_retrieval

In [63]:
sL1 = level1.isel(along=0, across=0)
sN = noise.isel(along=0, across=0)
ambiguity = 'sort_by_cost'

In [64]:
l2, lmout = ss.retrieval.level2.wind_current_retrieval(sL1, sN, gmf, ambiguity)

To Be Done find_initial_value
To Be Done - ambiguity_sort_by_cost
To Be Done - solve_ambiguity
To be done


In [65]:
sL1 = level1.isel(along=slice(0,2), across=slice(0,2))
sN = noise.isel(along=slice(0,2), across=slice(0,2))
ambiguity = 'sort_by_cost'

In [66]:
l2, lmout = ss.retrieval.level2.wind_current_retrieval(sL1, sN, gmf, ambiguity)

To Be Done find_initial_value
To Be Done - ambiguity_sort_by_cost
To Be Done - solve_ambiguity
To Be Done find_initial_value
To Be Done - ambiguity_sort_by_cost
To Be Done - solve_ambiguity
To Be Done find_initial_value
To Be Done - ambiguity_sort_by_cost
To Be Done - solve_ambiguity
To Be Done find_initial_value
To Be Done - ambiguity_sort_by_cost
To Be Done - solve_ambiguity


ValueError: cannot unstack dimensions that do not have exactly one multi-index: ('z',)

In [ ]:
l2

In [ ]:
lmout

In [ ]:
geo

In [40]:
L1stack

<xarray.Dataset>
Dimensions:              (Antenna: 4, z: 30)
Coordinates:
    CentralWavenumber    int64 270
    CentralFreq          float64 1.35e+10
    IncidenceAngleImage  (Antenna, z) int64 30 30 30 30 30 30 ... 30 30 30 30 30
    AntennaAzimuthImage  (Antenna, z) int64 45 45 45 45 45 ... 135 135 135 135
    Polarization         (Antenna, z) <U2 'VV' 'VV' 'VV' 'VV' ... 'VV' 'VV' 'VV'
  * Antenna              (Antenna) <U4 'Fore' 'MidV' 'MidH' 'Aft'
  * z                    (z) object MultiIndex
  * across               (z) int64 0 0 0 0 0 0 1 1 1 1 1 ... 3 3 3 3 4 4 4 4 4 4
  * along                (z) int64 0 1 2 3 4 5 0 1 2 3 4 ... 2 3 4 5 0 1 2 3 4 5
Data variables:
    Sigma0               (Antenna, z) float64 0.07128 0.07128 ... 0.1668 0.1668
    WASV                 (Antenna, z) float64 0.3227 0.3227 ... -1.498 -1.498
    RVL                  (Antenna, z) float64 0.06364 0.06364 ... -0.5313

In [39]:
L1stack.unstack(dim='z')

<xarray.Dataset>
Dimensions:              (across: 5, along: 6, Antenna: 4)
Coordinates:
  * across               (across) int64 0 1 2 3 4
  * along                (along) int64 0 1 2 3 4 5
    CentralWavenumber    int64 270
    CentralFreq          float64 1.35e+10
    IncidenceAngleImage  (Antenna, across, along) int64 30 30 30 30 ... 30 30 30
    AntennaAzimuthImage  (Antenna, across, along) int64 45 45 45 ... 135 135 135
    Polarization         (Antenna, across, along) <U2 'VV' 'VV' ... 'VV' 'VV'
  * Antenna              (Antenna) <U4 'Fore' 'MidV' 'MidH' 'Aft'
Data variables:
    Sigma0               (Antenna, across, along) float64 0.07128 ... 0.1668
    WASV                 (Antenna, across, along) float64 0.3227 ... -1.498
    RVL                  (Antenna, across, along) float64 0.06364 ... -0.5313

In [ ]:
level1

In [6]:
ds = xr.Dataset(
    data_vars=dict(
        mydata = ( ['across', 'along'], [[0,1],[2,3]])
    )
)
stacked = ds.stack(z=("across", "along"))
newlist = [None] * stacked.z.size
for ii, zindex in enumerate(stacked.z.data):
    newlist[ii] = stacked.mydata.sel(z=zindex)
newds = xr.concat(newlist, dim='z')
newds['z'] = stacked.z
print('xarray version:' + xr.__version__)
newds.unstack(dim='z')

xarray version:0.16.0


<xarray.DataArray 'mydata' (across: 2, along: 2)>
array([[0, 1],
       [2, 3]])
Coordinates:
  * across   (across) int64 0 1
  * along    (along) int64 0 1

In [96]:
ds = xr.Dataset(
    data_vars=dict(
        mydata = ( ['across', 'along'], [[0,1],[2,3]])
    )
)
stacked = ds.stack(z=("across", "along"))
newlist = [None] * stacked.z.size
for ii, zindex in enumerate(stacked.z.data):
    newlist[ii] = stacked.mydata.sel(z=zindex)
newds = xr.concat(newlist, dim='z')
newds['z'] = stacked.z
# newds = newds.reset_index('z')
print('xarray version:' + xr.__version__)
newds.unstack(dim='z')

xarray version:2022.11.0


ValueError: cannot unstack dimensions that do not have exactly one multi-index: ('z',)

In [74]:
stacked

<xarray.Dataset>
Dimensions:  (z: 4)
Coordinates:
  * z        (z) object MultiIndex
  * across   (z) int64 0 0 1 1
  * along    (z) int64 0 1 0 1
Data variables:
    mydata   (z) int64 0 1 2 3

In [75]:
ds

<xarray.Dataset>
Dimensions:  (across: 2, along: 2)
Dimensions without coordinates: across, along
Data variables:
    mydata   (across, along) int64 0 1 2 3

In [7]:
stacked.unstack(dim='z')

<xarray.Dataset>
Dimensions:  (across: 2, along: 2)
Coordinates:
  * across   (across) int64 0 1
  * along    (along) int64 0 1
Data variables:
    mydata   (across, along) int64 0 1 2 3